In [1]:
!pip install torch
!pip install stable-baselines3[extra]
!pip install gymnasium
!pip install 'gymnasium[atari]'
!pip install ale-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.2 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446667 sha256=9e50dfb4da6db35f0b1d05ceb9c6d2fef762d0461354cccb51ebb77b5b30c594
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
  Attempting uninstall: shimmy
    Found existing installation: Shimmy 1.3.0
    Uninstalling Shimmy-1.3.0:
      Successfully uninstalled Shimmy-1.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.16.10 require

In [2]:
import os
import gymnasium as gym

import ale_py 

import os

import torch

from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.evaluation import evaluate_policy

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/util.py:55: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs; see https://jax.readthedocs.io/en/latest/aot.html. For example, replace xla_computation(f)(*xs) with jit(f).lower(*xs).compiler_ir('hlo'). See CHANGELOG.md for 0.4.30 for more examples.
  from jax import xla_computation as _xla_computation


In [3]:
gym.register_envs(ale_py)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
environment_name = 'MsPacman-v4'

In [5]:
env = gym.make(environment_name, render_mode='rgb_array')

In [6]:
env.metadata['render_fps'] = 30  # Définir le FPS de rendu

In [7]:
episodes = 1
for episode in range(1, episodes+1):
    state = env.reset()
    terminated = False
    score = 0

    while not terminated:
        env.render() 
        action = env.action_space.sample()
        observation, reward, terminated, truncated, info = env.step(action)
        score += reward
    print(f'Episode: {episode}, Score: {score}')
env.close()

Episode: 1, Score: 280.0


In [8]:
env.action_space

Discrete(9)

In [9]:
env.observation_space

Box(0, 255, (210, 160, 3), uint8)

# Training DQN

In [10]:
environment_name = 'MsPacman-v4'

In [11]:
env = make_atari_env(environment_name, n_envs=1, seed=0)

In [12]:
# Empilement des frames pour prendre en compte l'historique des observations
env = VecFrameStack(env, n_stack=4)


In [13]:
log_path = os.path.join('Training', 'Logs','dqn_pacman_tensorboard')

In [14]:
# Vérifier si un GPU est disponible
print(f"GPU disponible : {torch.cuda.is_available()}")
print(f"Nom du GPU : {torch.cuda.get_device_name(0)}")

GPU disponible : True
Nom du GPU : Tesla P100-PCIE-16GB


In [15]:
# Initialisation du modèle DQN
model = DQN(
    'CnnPolicy',  # Utilisation d'un CNN pour traiter les images
    env,
    verbose=1,
    learning_rate=1e-4,
    buffer_size=100000,  # Taille de la mémoire de replay
    learning_starts=10000,  # Nombre de steps avant de commencer l'apprentissage
    batch_size=32,
    tau=1.0,  # Paramètre pour la mise à jour du réseau cible
    gamma=0.99,  # Facteur de discount
    train_freq=4,  # Fréquence de mise à jour du réseau
    target_update_interval=1000,  # Fréquence de mise à jour du réseau cible
    exploration_fraction=0.1,  # Fraction de l'exploration
    exploration_initial_eps=1.0,  # Exploration initiale
    exploration_final_eps=0.01,  # Exploration finale
    tensorboard_log=log_path  # Dossier pour les logs TensorBoard
)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [16]:
# Entraînement du modèle
model.learn(total_timesteps=1000000, log_interval=100)

Logging to Training/Logs/dqn_pacman_tensorboard/DQN_1


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


----------------------------------
| rollout/            |          |
|    ep_len_mean      | 674      |
|    ep_rew_mean      | 365      |
|    exploration_rate | 0.946    |
| time/               |          |
|    episodes         | 100      |
|    fps              | 311      |
|    time_elapsed     | 17       |
|    total_timesteps  | 5417     |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 691      |
|    ep_rew_mean      | 385      |
|    exploration_rate | 0.89     |
| time/               |          |
|    episodes         | 200      |
|    fps              | 288      |
|    time_elapsed     | 38       |
|    total_timesteps  | 11099    |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0552   |
|    n_updates        | 274      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean    

# Save & Reload Model

In [17]:
dqn_path = os.path.join('Training', 'Saved Models', 'dqn_pacman')

In [18]:
# Sauvegarde du modèle
model.save(dqn_path)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:278: UserWarning: Path 'Training/Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [19]:
# Évaluation du modèle
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward}")

Mean reward: 1833.0 +/- 423.1796308897677


In [20]:
# Fermeture de l'environnement
env.close()